In [1]:
from libs.data_manipulation import *
from libs.feature_generation import *
from libs.dim_reduction import *
from libs.football_plots import *
from libs.clustering import *
from libs.convex_hull import *
from libs.weight_generator import *
from scipy.stats import wasserstein_distance_nd


/home/martin/.local/lib/python3.10/site-packages/requests/__init__.py:109: RequestsDependencyWarning: urllib3 (2.2.3) or chardet (None)/charset_normalizer (2.1.0) doesn't match a supported version!
  warnings.warn(


In [2]:
england_df = compile_team_tracking_data("data", "England")


In [3]:
def normalize_positions_with_ball(df):
    # Create copies of ball x and y columns for both home and away teams
    df_normalized = df.copy()
    
    # Normalize 'home' team player positions
    home_columns_x = [col for col in df.columns if col.startswith('home') and col.endswith('_x')]
    home_columns_y = [col for col in df.columns if col.startswith('home') and col.endswith('_y')]
    
    for x_col, y_col in zip(home_columns_x, home_columns_y):
        df_normalized[x_col] = df[x_col] - df['ball_x']
        df_normalized[y_col] = df[y_col] - df['ball_y']
    
    # Normalize 'away' team player positions
    away_columns_x = [col for col in df.columns if col.startswith('away') and col.endswith('_x')]
    away_columns_y = [col for col in df.columns if col.startswith('away') and col.endswith('_y')]
    
    for x_col, y_col in zip(away_columns_x, away_columns_y):
        df_normalized[x_col] = df[x_col] - df['ball_x']
        df_normalized[y_col] = df[y_col] - df['ball_y']
    
    return df_normalized

In [4]:
one_match = extract_one_match(england_df,3).iloc[::48]
one_match_oringal = one_match

identified_corner_timestamp = 4432.84

In [5]:



identified_corner_df = one_match[one_match["Time [s]"] == identified_corner_timestamp]

func = lambda x : 1/x


inverse_identified_corner_weights = calculate_weights(identified_corner_df, fun = func)
inverse_distance_list = calculate_weights(one_match, fun= func) #Inverse proportionality to distance

In [6]:
from scipy.stats import wasserstein_distance_nd
one_match = normalize_positions_with_ball(one_match)
coordinates = one_match.filter(regex ="^home").to_numpy()


identified_corner_coordinates_numpy = identified_corner_df.to_numpy()
identified_corner_coordinates = [list(zip(row[~np.isnan(row)][::2],row[~np.isnan(row)][1::2])) for row in coordinates]
coordinates_zipped = [list(zip(row[~np.isnan(row)][::2],row[~np.isnan(row)][1::2])) for row in coordinates]


In [7]:
print(len(inverse_identified_corner_weights[0]))


11


In [8]:

#Get closest situations
distances = []
i = 0
for weights, coordinates in zip(inverse_distance_list, coordinates_zipped):
    i+=1
    if(not np.isnan(np.sum(weights)) and len(weights) == 11 and len( coordinates) == 11):
        
        distances.append((wasserstein_distance_nd(identified_corner_coordinates[0], coordinates, u_weights= inverse_identified_corner_weights[0], v_weights=weights), i))

In [9]:
indices_and_distances = sorted(distances, key = lambda t: t[0])
indices = [index for _,index in indices_and_distances]
print(indices)

[9771, 9243, 8219, 4043, 9769, 9750, 2297, 9756, 724, 9891, 9751, 9909, 3489, 3462, 746, 3428, 9757, 9753, 9142, 9755, 3488, 9754, 9758, 140, 8755, 3490, 3492, 3491, 5121, 3487, 9341, 4079, 3494, 3481, 723, 4067, 3493, 8963, 3486, 953, 9870, 8032, 3495, 3482, 4068, 3485, 9871, 1917, 1922, 3480, 1921, 9498, 8954, 722, 1920, 3496, 1918, 9759, 1919, 621, 8854, 9913, 9912, 3497, 2298, 4082, 4083, 3500, 3498, 9872, 3501, 142, 3479, 617, 3502, 3477, 3483, 10041, 3499, 3484, 719, 9892, 1923, 10040, 9404, 952, 4080, 1916, 1926, 4044, 10009, 1927, 9235, 8220, 9866, 9141, 9760, 9492, 3073, 10008, 4283, 4223, 4086, 8956, 616, 1928, 1924, 718, 9873, 1929, 1925, 6078, 138, 1569, 7379, 1915, 7939, 6007, 9493, 3463, 7405, 139, 9807, 2280, 3892, 9411, 6006, 1944, 8955, 6085, 1914, 458, 3891, 9727, 1912, 9971, 4112, 7489, 721, 2281, 1911, 726, 9405, 9230, 7055, 3478, 8361, 727, 1913, 3369, 2707, 1471, 7486, 1605, 4118, 9137, 2708, 1381, 8481, 9767, 5833, 7651, 574, 3282, 868, 6077, 8568, 9499, 1382, 32

In [10]:
situations = one_match.iloc[indices[:10]].index.to_numpy()


In [11]:

generate_pitches_from_start_indices(np.concatenate((situations,identified_corner_df.index.to_numpy())), one_match_oringal, "Ripleysk_k_plots",1,48)

/home/martin/anaconda3/envs/DBU/lib/python3.10/site-packages/matplotlib/quiver.py:640: RuntimeWarning: Mean of empty slice.
  amean = a[~self.Umask].mean()
/home/martin/.local/lib/python3.10/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
